In [ ]:
import os
import numpy as np
import pathlib
import glob
import warnings
import cv2
from pyometiff import OMETIFFReader,OMETIFFWriter
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
base_dir = "./"
ome_dir_path: str = "path/to/dir"
output_dir_path: str = "path/to/dir"
ome_dir: str = pathlib.Path(ome_dir_path)
output_dir: str = pathlib.Path(output_dir_path)
ome_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Gather all the OME-TIFFs in list
ome_tiffs = list(ome_dir.glob("*.ome.[tif tiff]*"))
image_data = [] # Contains Nested List for each image and slice 
flist = [] # Contains fpath

# Convert all the OME-TIFFs to Single Channel TIFFs
for ome_tiff in ome_tiffs:
    img_fpath = pathlib.Path(ome_tiff)
    reader = OMETIFFReader(fpath=img_fpath)
    img_array, metadata, xml_metadata = reader.read()
    name = ome_tiff.__str__().split("\\")[2]
    dir_path = os.path.join(output_dir, name)
    if not os.path.exists(dir_path):
        os.mkdir(dir_path,)
    flist.append(dir_path)
    #Add Indidial Tiff images to dataset 
    temp = []
    for image in img_array:
        temp.append(image)
    #image_data.append(((resize(img_array[5], (512, 512))),(resize(img_array[7], (512, 512))),(resize(img_array[0], (512, 512)))))
    image_data.append(temp)

In [ ]:
def detect_shapes(image):
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Threshold the image to obtain binary image
    _, thresh = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Iterate through each contour
    for i, contour in enumerate(contours):
        # Get the bounding box coordinates
        x, y, w, h = cv2.boundingRect(contour)
        
        # Determine shape based on aspect ratio of bounding box
        aspect_ratio = float(w) / h
        shape = "Unknown"
        if aspect_ratio >= 0.95 and aspect_ratio <= 1.05:
            shape = "Square"
        else:
            shape = "Rectangle"
        
        # Draw the bounding box and label the shape
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, shape, (x, y - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Print the x and y coordinates of the bounding box
        print(f"Shape {i+1}: {shape}, Bounding Box: ({x}, {y}), ({x + w}, {y + h})")

    # Display the image with detected shapes
    cv2.imshow("Detected Shapes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the function with an image
image_path = image_data[0][4]
detect_shapes(image_path)

In [ ]:

def detect_shapes(image, min_size):
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Threshold the image to obtain binary image
    _, thresh = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)
   
    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to store the areas where shapes will be removed
    mask = np.zeros_like(image)
    
    min_area = 300000
    # Iterate through each contour
    for contour in contours:
        # Get the bounding box coordinates
        approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
        x, y, w, h = cv2.boundingRect(contour)
        print(f"width = {w}, height = {h}")
        print(f"{w * h} is the Area")
        area = w * h
        aspect_ratio = float(w) / h
        if (w > min_size or h > min_size) and (area >= min_area):  # Aspect ratio close to 1 indicates a square and check for size
            # Draw the contour on the mask
            if len(approx) == 4:
                cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1)


        #elif (w > min_size or h > min_size) and (area >= min_area):
            # For non-large rectangles or non-rectangular contours, draw a filled rectangle on the mask
            #cv2.drawContours(mask, (x, y), (x + w, y + h), (255, 255, 255), -1)
        

    
    # Invert the mask
    mask_inv = cv2.bitwise_not(mask)
    
    # Apply the mask to the original image to remove the shapes
    image_removed = cv2.bitwise_and(image, image, mask=mask_inv)
    # Display the image with detected shapes removed
    
    return image_removed,mask_inv



In [ ]:
min_size = 580
for i,image in enumerate(image_data):
    for j,fov in enumerate(image):
        im, mask_inv = detect_shapes(fov, min_size)
        cv2.imwrite(f"{flist[i]}/img{j}.png",im)
        cv2.imwrite(f"{flist[i]}/mask{j}.png",mask_inv)